# To Do

- Implement n_i^eq + relaxation 
- Implement intial densities  (now just set to 0), make sure that upper and lower boundary are set to 0.
- Implement input and output boundaries (bounch has been done already
- Implement pressure gradient (hence for in x diretion)
- Implement visualization 


In [147]:
import numpy as np
from sys import exit 
from types import SimpleNamespace

def boltzmann_sim(self):
    """
    
    Simulation of the boltzmann lattice
    
    
    """
    input_check(self)
    
    sim, par = initialization(self)
    par = shift_n(self, par)
    par = boundary_bounch(self, par)
    
    return par

def input_check(self):
    if (self.L/self.res)%1 !=0 or (self.W/self.res)%1 != 0:
        exit('Choose width, height and res such that an integer ammount of points is generated.')

def initialization(self):
    """ Initializes arrays. 
    
    Parameters
    ----------
    self : NameSpace
        simulation constants 
        
    Returns
    -------
    par: NameSpace
        containing initialized parameter arrays
    
    """
    
    # Simulation parameters 
    par = SimpleNamespace()
    
    # Cartesian grid coordinates 
    self.grid_coord = np.reshape(np.meshgrid(np.linspace(0, self.L, self.L_n), np.linspace(0, self.W, self.W_n)), (2,-1))
    
    # Integer grid coordinates
    self.grid_int_coord = np.reshape(np.meshgrid(range(self.L_n), range(self.W_n)), (2,-1))
    
    # Density array
    par.n = np.zeros((self.L_n, self.W_n, len(self.e)), dtype = float)
    
    return self, par

def shift_n(self, par):
    """ Shifts densities according to their unit vectors. 
    
    Parameters
    ----------
    self : NameSpace
        simulation constants 
    par : NameSpace
        simulation parameters 
    
    Returns
    -------
    par: NameSpace
        updated simulation parameters
    
    """
    
    for i in range(len(self.e)):
        par.n[i] = np.roll(par.n[i], self.e[i])
    
    return par

def boundary_bounch(self, par):
    """
    
    Takes the upper boundary and the lower boundary of a certain density vector. 
    Mirrors this vector and assigns the density accordingly. 
    
    Parameters
    ----------
    self : NameSpace
        simulation constants 
    par : NameSpace
        simulation parameters 
    
    Returns
    -------
    par: NameSpace
        updated simulation parameters
    
    """    
    
    # To keep track of exchanged boundaries
    exchanged = np.zeros((len(self.e),), dtype = bool)
    
    for i in range(len(self.e)):
        if exchanged[i]:
            e_inv = -1 * self.e[i]
            j = np.argwhere(np.sum(e_inv == sim.e, axis = 1) == 2)[0]
            
            # Select upper and lower boundary 
            bd_1 = par.n[i, [0, self.W_in]]
            bd_2 = par.n[j, [0, self.W_in]]
            
            # Exchange densities accordingly 
            par.n[i, [0, self.W_in]] = bd_2
            par.n[j, [0, self.W_in]] = bd_1
            exchanged[i, j] = True
            
    return par 

In [145]:
import numpy as np
from types import SimpleNamespace

sim = SimpleNamespace()
sim.res = 0.2
sim.L, sim.W = 4, 1
sim.L_in, sim.W_in = int(sim.L/sim.res), int(sim.L/sim.res)  
sim.L_n,  sim.W_n  = sim.L_in + 1, sim.W_in + 1

sim.e = np.array([[0,0], [1,0], [1,1], [0,1], [-1,1], [-1,0], [-1,-1], [0,-1], [1,-1]])

sim.nu = 1

par = boltzmann_sim(sim)

